# ML model generation

This notebook creates and trains a simple fully connected model using MNIST dataset. At the end, the model is exported as `model.bin`.

In [220]:
#|export
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import struct
import os

import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA (GPU) is available.")
else:
    device = torch.device("cpu")
device

device(type='cpu')

In [3]:
#|export
import torchvision
from torchvision.datasets.utils import download_and_extract_archive

dir_path = "data"
os.makedirs(dir_path, exist_ok=True)

# URL for Imagenette (Full size)
url = "https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz"

# Download and extract the dataset
download_and_extract_archive(url, download_root=dir_path, extract_root=dir_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1557161267/1557161267 [01:32<00:00, 16777967.29it/s]


Extracting data/imagenette2.tgz to data


In [16]:
#|export
def generate_dataloader(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to fit ResNet input size
        transforms.ToTensor(),
    ])
    
    train_dataset = ImageFolder(root=f"data/imagenette2/train", transform=transform)
    test_dataset = ImageFolder(root=f"data/imagenette2/val", transform=transform)

    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             num_workers=torch.get_num_threads())
    testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=torch.get_num_threads())
    
    return trainloader, testloader

In [5]:
#|export
def test_model(model, testloader):
    loss_fn = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        vloss = 0.
        correct = 0.

        # Fetch the first batch
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            vloss += loss_fn(outputs, targets).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted==targets).sum().item()
    
    return vloss/len(testloader),  correct/len(testloader.dataset)


def train_model(model):  
    # training
    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=0.001)
    trainloader, testloader = generate_dataloader()

    for epoch in range(1):

        model.train()
        tloss = 0
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            opt.zero_grad()
            out = model(inputs)
            loss = loss_fn(out, targets)
            loss.backward()
            tloss += loss.item()
            opt.step()

        tloss = tloss/len(trainloader)
        vloss, correct = test_model(model, testloader)

        print('LOSS train {} valid {} accuracy {:.5f}'.format(tloss, vloss, correct))
    torch.save(model, "model.pt")

In [ ]:
# swap the output layer and freeze all other layers
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
for p in model.parameters():
    p.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 10)
model.to(device)

In [ ]:
# fine-tuning
#FIX find optimal hyperparameters + maybe unfreeze after training final layer and then train the entire model
train_model(model)

In [ ]:
_, testloader = generate_dataloader()
loss, acc = test_model(model, testloader)
print("Accuracy: {:.2f} %".format(100*acc))